In [10]:
import json,os
import numpy as np
import pandas as pd

### Cargo datos

In [11]:
ligue2team2rank = json.load( open("../data/league2team2rank.json", "r") )

In [12]:
matches = pd.read_csv("../data/matches.csv")
matches.head(2)

,match,home,away,winner,score_home,score_away,label,liga
0,2500089,1646,1659,1659,1,2,"Burnley - AFC Bournemouth, 1 - 2",England
1,2500090,1628,1627,1628,2,0,"Crystal Palace - West Bromwich Albion, 2 - 0",England


In [13]:
metrics = pd.read_csv("../data/estilos.csv")
metrics.head(2)

,Unnamed: 0,S1,S2,S3,S4,match,team1,liga
0,0,1.193054,-2.360068,-0.858711,-2.101015,2499719,1609,England
1,1,-3.041211,0.444480,0.290051,1.924527,2499719,1631,England


### Armo redes

Primero cruzo los datos por match

In [14]:
df0 = pd.merge(metrics, matches[['match', 'home', 'away', 'winner', 'score_home','score_away']], on='match', how='left')

df0.head(2)

,Unnamed: 0,S1,S2,S3,S4,match,team1,liga,home,away,winner,score_home,score_away
0,0,1.193054,-2.360068,-0.858711,-2.101015,2499719,1609,England,1609,1631,1609,4,3
1,1,-3.041211,0.444480,0.290051,1.924527,2499719,1631,England,1609,1631,1609,4,3


In [15]:
metricas = ['S1','S2','S3','S4']

ligas = ['England','France','Germany','Italy','Spain']



for liga in ligas:

    carpeta = f'{liga}'
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)


    team2rank = ligue2team2rank[liga]
    df = df0[ df0['liga']==liga ]


    for metrica in metricas:

        d={}

        def f(row):

            # tomo datos
            team1 = str(row.team1)
            nodo1 = team2rank[team1]

            if row.team1==row.home:
                team2=str(row.away)
            else:
                team2=str(row.home)

            nodo2 = team2rank[team2]

            w = row[metrica] 

            # armo red
            try:
                d[(nodo1,nodo2)]+=w
            except:
                d[(nodo1,nodo2)]=w
            

        df.agg( lambda x: f(x), axis=1 )

        # armo red
        Nmax = max(elemento for tupla in d for elemento in tupla)
        red = []

        for i in range(1,Nmax):

            for j in range(i+1, Nmax+1):

                fij = d[(j,i)] - d[(i,j)] 

                red.append( [i,j, fij] )

        #guardo data
        arx_out = f'{liga}/red_{metrica}.dat'



        np.savetxt(
            arx_out,               
            red,               
            fmt='%d,%d,%.4e',         
            delimiter=',', 
            header='n1,n2,fij',
            comments=''
        )

